# Basic usage of the cosmoDC2 extragalactic catalog and photoz add-on catalog - Weak gravitational lensing shear by dark matter halo

### Compute the excess surface density arround clusters using photometrical redshift in cosmoDC2 extragalactic catalog

#### This notebook can be run at NERSC or CC-IN2P3 where the DESC DC2 products are stored. You need to be a DESC member to be able to access those.

This notebook will show how to use the photometric redshift (photo-z) add-on columns of cosmoDC2 extragalactic catalog in order to estimate the average excess surface density around cluster. After going through this notebook, you should be able to:
- 1. load a cosmoDC2 dark matter halo catalog,
- 2. load and access the cosmoDC2 extragalctic catalog,
- 3. load and access the cosmoDC2 photo-z add-on columns (a detailled version of this step was edited by Sam Shmidt and is available at https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/extragalactic_gcr_photoz_catalogs.ipynb),
- 4. Compute the excess surface density estimator using phorometric redshift catalogs.

In [1]:
import numpy as np
import os
import GCRCatalogs
import matplotlib.pyplot as plt
import scipy.integrate

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, join
import GCRCatalogs
os.environ['CLMM_MODELING_BACKEND'] = 'ccl' # here you may choose ccl, nc (NumCosmo) or ct (cluster_toolkit)
import clmm 
import healpy

# 1. Prepare a dark matter halo from cosmoDC2

In [ ]:
extragalactic_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')

In order to speed up the extraction of dark matter halo/galaxy quantities, we select a dark matter halo within a specified healpix pixel. 

In [ ]:
healpix_pixels = extragalactic_cat.get_catalog_info()['healpix_pixels']

In [ ]:
healpix_selected = healpix_pixels[4]
print('The selected healpix pixel is ' + str(healpix_selected))

We extract the dark matter halos located in this specified healpix pixel with some property cut (mass and redhift range). We select the most massive one in the sample.

In [ ]:
# get list of massive halos in a given redshift and mass range
mmin = 1.e14 # Msun
zmin = 0.3
zmax = 0.4
massive_halos = extragalactic_cat.get_quantities(['halo_mass','hostHaloMass','redshift','ra', 'dec', 'halo_id'],
                                                 filters=[f'halo_mass > {mmin}','is_central==True',
                                                          f'redshift>{zmin}', f'redshift<{zmax}'], native_filters=['healpix_pixel == ' + str(healpix_selected)])
N_cl = len(massive_halos['halo_mass'])
print(f'There are {N_cl} clusters in this mass and redshift range')

In [ ]:
select = massive_halos['halo_mass'] == np.max(massive_halos['halo_mass'])
ra_cl = massive_halos['ra'][select][0]
dec_cl = massive_halos['dec'][select][0]
z_cl = massive_halos['redshift'][select][0]
mass_cl =massive_halos['halo_mass'][select][0]
id_cl = massive_halos['halo_id'][select][0]

In [ ]:
print (f'The most massive cluster is halo {id_cl} in ra = {ra_cl:.2f} deg, dec = {dec_cl:.2f} deg, z = {z_cl:.2f}, with mass = {mass_cl:.2e} Msun')

# 2. Extract background galaxy catalog with location and shape properties

The galaxy shapes are available in the cosmoDC2 extragalaxtic catalog, and photometric related inforamtions are available separetly in add-on columns. We first extract the background galaxy shapes for galaxies within a window of 0.6 degrees arround the cluster position. As the redshift cut to identify background galaxies is applied to mean photometric redshift of galaxies ($\langle z_{\rm gal} \rangle_{\rm photoz} > z_{\rm l} + 0.1$), where $z_{\rm l}$ is the redshift of the lens dark matter halo. we then extract the all galaxy information in this healpix pixel to apply futher cuts.

### Apply coordinates, redshift and magnitude cuts to select backgroud galaxies around the cluster

- Box of 0.6 deg around the cluster center
- Galaxies with mag_i < 25

In [ ]:
ra_min, ra_max = ra_cl - 0.3, ra_cl + 0.3
dec_min, dec_max = dec_cl - 0.3, dec_cl + 0.3
mag_i_max = 25

We load the catalog in the contiguous healpix pixels arround the cluster position corresponding to a window O.6 deg (we use the `healpy` package to find the correspondance healpix-position)

In [ ]:
n_points = 300
ra_random, dec_random = np.random.random(n_points)*(ra_max - ra_min) + ra_min, np.random.random(n_points)*(dec_max - dec_min) + dec_min
contiguous_healpix_pixels = np.unique(healpy.ang2pix(32, ra_random, dec_random, nest=False, lonlat=True))
print('The contiguous healpix pixels arround the cluster position are '+ str(contiguous_healpix_pixels))

To compute the excess surface density arround the dark matter halo, we extract the weak lensing quantities for each background galaxies:
- complex shear componenent `'shear_1'` & `'shear_2'`
- convergence `'convergence'`
- intrinsic ellipticity components `'ellipticity_1_true'` & `'ellipticity_2_true'`

In [ ]:
%%time
coord_filters = ['ra >= {}'.format(ra_min),'ra < {}'.format(ra_max),'dec >= {}'.format(dec_min),'dec < {}'.format(dec_max)]
z_filters = ['redshift >= 0']
mag_filters = ['mag_i < {}'.format(mag_i_max)]
gal_cat = extragalactic_cat.get_quantities(['galaxy_id', 
                                            'ra', 'dec',
                                            'shear_1', 'shear_2', 
                                            'ellipticity_1_true', 'ellipticity_2_true',
                                            'redshift', 'convergence', 'halo_id'],
                                            filters=(coord_filters + z_filters + mag_filters), 
                                            native_filters=['healpix_pixel == ' + str(healpix_selected)])
print(str(len(gal_cat['galaxy_id'])) +' saved galaxies')

We compute the obseved galaxy ellipticities using shear, convergence and intrinsic ellipticities (we use the functionalities of the `CLMM` package):

$$
\epsilon^{\rm obs} =
        \frac{\epsilon^{\rm int} + g}{1 + g^*\epsilon^{\rm int}}$$ where: $$g = \frac{\gamma}{1-\kappa}$$

In [ ]:
gal_cat['e1'] = clmm.utils.compute_lensed_ellipticity(gal_cat['ellipticity_1_true'], 
                                                      gal_cat['ellipticity_2_true'], 
                                                      gal_cat['shear_1'], gal_cat['shear_2'], 
                                                      gal_cat['convergence'])[0]

gal_cat['e2'] = clmm.utils.compute_lensed_ellipticity(gal_cat['ellipticity_1_true'], 
                                                      gal_cat['ellipticity_2_true'], 
                                                      gal_cat['shear_1'], 
                                                      gal_cat['shear_2'], 
                                                      gal_cat['convergence'])[1]

In [ ]:
gal_cat['z'] = gal_cat['redshift']

In [ ]:
gal_cat = Table(gal_cat)

The member galaxy catalog for the selected clusters :

In [ ]:
member_gal_catalog = gal_cat[gal_cat['halo_id'] == id_cl]

# 3. Extract photometrical related informations

The full details of the photo-z quantities in the add-on catalog `'cosmoDC2_v1.1.4_small_with_photozs_v1'` are given in the notebook https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/extragalactic_gcr_photoz_catalogs.ipynb by Sam Schmidt. This catalog contains photometric redshifts that were computed with the BPZ template-based code (https://ui.adsabs.harvard.edu/abs/2000ApJ...536..571B/abstract). In this notebook we show the estimation of the excess surface density using BPZ redshift. FlexZBoost photometric redshift are also avaliable by taking the catalog `'cosmoDC2_v1.1.4_small_with_photozs_flexzboost_v1'`. For weak lensing purposes, we extract for each background galaxies in the above sample:
- the probability density function `'photoz_pdf'`,
- the mean photometric redshift `'photoz_mean'`.

Let's note that the add-on catalog does not support geometrical cut (in ra, dec, redshift). We first extract the photo-z informations in the corresponding contiguous healpix pixels, we then match with the galaxy sample.

In [ ]:
%%time
cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small_with_photozs_v1')
z_bins = cat.photoz_pdf_bin_centers

In [ ]:
%%time
photoz_cat = cat.get_quantities(['photoz_pdf','photoz_mean','redshift', 'galaxy_id'],
                            native_filters=['healpix_pixel=='+str(contiguous_healpix_pixels[0])])

We use package to match with the galaxy sample.

In [ ]:
z_min = z_cl + 0.1

In [ ]:
tab_astropy = Table()
tab_astropy['galaxy_id'] = photoz_cat['galaxy_id']
tab_astropy['photoz_pdf'] =  photoz_cat['photoz_pdf']
tab_astropy['photoz_mean'] =  photoz_cat['photoz_mean']

In [ ]:
cat_complete = join(tab_astropy, gal_cat, keys = 'galaxy_id')

In [ ]:
axis_select_background = [0 if z_bins[i] < z_min + 0.1 else 1 for i in range(len(z_bins))]

In [ ]:
m = np.zeros([len(cat_complete), len(z_bins)])
for i in range(len(cat_complete)):
    m[i,:] = np.array(cat_complete['photoz_pdf'][i])

In [ ]:
integral_pdf_behind = scipy.integrate.simps(m * axis_select_background, x = z_bins, axis = 1)
norm_pdf = scipy.integrate.simps(m, x = z_bins, axis = 1)

In [ ]:
cat_complete['background_probability'] = integral_pdf_behind/norm_pdf
cat_complete['photoz_pdf_norm'] = norm_pdf

In [ ]:
member_gal_photoz_catalog = cat_complete[np.isin(cat_complete['galaxy_id'],member_gal_catalog['galaxy_id'])]

### Selection of background galaxies:
We discuss briefly the effect of redhsift selection of background galaxies, by considering two different cases:
- `cat_photoz`: 
The background galaxies can be selected by considering that the mean photometric redshift veryfies $\langle z \rangle_{\rm photoz} > z_{\rm l} + 0.1$,
- `cat_true`: 
The background galaxies are selected with $z_{\rm cosmoDC2} > z_{\rm l} + 0.1$.
- `cat_probability`: 
The background galaxies are selected with $\int_{z_{\rm DM + 0.1}}^{+\infty} dz p(z)  > 0.95$.

In [ ]:
cat_true = cat_complete[cat_complete['redshift'] > z_min]
cat_photoz = cat_complete[cat_complete['photoz_mean'] > z_min]
cat_probability = cat_complete[cat_complete['background_probability'] > 0.95]

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (20,8))
plt.rcParams['axes.linewidth'] = 2
ax[0].tick_params(axis='both', which = 'both', labelsize= 15)
ax[0].scatter(cat_photoz['photoz_mean'], cat_photoz['background_probability'], label = r'cut $: \langle z \rangle_{\rm photoz}  > z_{\rm DM} + 0.1$')
ax[0].scatter(cat_probability['photoz_mean'], cat_probability['background_probability'], label =  r'cut $: P(z > z_{\rm DM} + 0.1)  > 0.95$')
ax[0].vlines(z_cl, 0, 1, color = 'magenta', linestyle = '-', label = 'cluster redshift', zorder = 1000)
ax[0].legend(frameon = True, framealpha = 1, fontsize = 15, loc = 'lower right')
ax[0].set_xlabel(r'$\langle z \rangle_{\rm photoz}$', fontsize = 30)
ax[0].set_ylabel(r'$P(z > z_{\rm DM} + 0.1)$', fontsize = 30)

ax[1].tick_params(axis='both', which = 'both', labelsize= 15)
ax[1].scatter(cat_photoz['redshift'], cat_photoz['background_probability'], label = r'cut $: \langle z \rangle_{\rm photoz}  > z_{\rm DM} + 0.1$')
ax[1].scatter(cat_probability['redshift'], cat_probability['background_probability'], label =  r'cut $: P(z > z_{\rm DM} + 0.1)  > 0.95$')
ax[1].vlines(z_cl, 0, 1, color = 'magenta', linestyle = '-', label = 'cluster redshift', zorder = 1000)
ax[1].legend(frameon = True, framealpha = 1, fontsize = 15, loc = 'lower right')
ax[1].set_xlabel(r'$z_{\rm cosmoDC2}$', fontsize = 30)
ax[1].set_ylabel(r'$P(z > z_{\rm DM} + 0.1)$', fontsize = 30)

We check the contamination level by counting the member galaxies after selection in each dataset:

In [ ]:
print(' ')
member_gal_cat_photoz = cat_photoz[np.isin(cat_photoz['galaxy_id'],member_gal_catalog['galaxy_id'])]
print(f'there are {len(member_gal_cat_photoz)} member galaxies in the cat_photoz dataset')
foreground_gal_photoz = len(cat_photoz['redshift'] < z_cl)
print(f'there are {foreground_gal_photoz} foreground galaxies in the cat_photoz dataset')
print(' ')
member_gal_cat_probability = cat_probability[np.isin(cat_probability['galaxy_id'],member_gal_catalog['galaxy_id'])]
print(f'there are {len(member_gal_cat_probability)} member galaxies in the cat_probability dataset')
print(' ')
member_gal_cat_true = cat_true[np.isin(cat_true['galaxy_id'],member_gal_catalog['galaxy_id'])]
print(f'there are {len(member_gal_cat_true)} member galaxies in the cat_true dataset')
print(' ')

From the figure above, and the count of contamination in each data set, we can see that the background galaxy selection based of mean photometric redshift induces a larger contamination from member galaxies in the background galaxy sample. 

In the figure below, we show the different dataset involved:
- the true redshift histogram in the selected healpix (orange)
- the true redshift histogram in the `cat_photoz` dataset (blue)
- the true redshift histogram in the `cat_true` dataset (dashed red)
- the individual photoz-pdf of each member galaxies (grey)

In [ ]:
pdf_member_galaxy_average = 0
for i, pdf in enumerate(member_gal_photoz_catalog['photoz_pdf']):
    pdf_member_galaxy_average = pdf_member_galaxy_average + pdf/member_gal_photoz_catalog['photoz_pdf_norm'][i]

In [ ]:
plt.figure(figsize = (12,8))
plt.rcParams['axes.linewidth'] = 2
bins = np.linspace(0, 3, 56)
plt.tick_params(axis='both', which = 'both', labelsize= 15)
plt.hist(gal_cat['redshift'], bins = bins, linewidth = 1, label = r'healpix $z_{\rm cosmoDC2}$ > 0',  color = 'orange', alpha = 0.5)
plt.hist(cat_photoz['redshift'], bins = bins, histtype = 'step', linewidth = 2, label = r'cut $: \langle z \rangle_{\rm photoz} > z_{\rm DM} + 0.1$', color = 'b')
plt.hist(cat_true['redshift'], bins = bins, linewidth = 4, histtype = 'step', label = r'cut $: z_{\rm cosmoDC2}  > z_{\rm DM} + 0.1$',color = 'r', linestyle = '--', zorder = 0)
plt.hist(cat_probability['redshift'], bins = bins, linewidth = 2, histtype = 'step', label = r'cut $: P(z > z_{\rm DM} + 0.1)  > 0.95$',color = 'g', linestyle = '--')
plt.vlines(z_cl, 0, 4000, color = 'k', linestyle = '--', label = 'cluster redshift', zorder = 1000)

plt.plot(z_bins, 500*pdf_member_galaxy_average/len(member_gal_photoz_catalog), '-k', label = 'average member galaxy photoz pdf')

plt.ylim(0, 4000)
plt.legend(frameon = False, fontsize = 20)
plt.xlabel(r'$z_{\rm cosmoDC2}$', fontsize = 30)
plt.ylabel(r'Histogram', fontsize = 30)
plt.title('Selection effect on background galaxies', fontsize = 20)
plt.savefig('/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Images/selection_effect.png', bbox_inches='tight', dpi=300)
plt.show()

We can check the percentage of foreground galaxies in the `cat_photoz` dataset that have been identified as background galaxies

In [ ]:
per_cent_cat_photoz = abs(100*(len(cat_photoz[cat_photoz['redshift'] < z_cl]))/len(cat_photoz))
per_cent_cat_probability = abs(100*(len(cat_probability[cat_probability['redshift'] < z_cl]))/len(cat_probability))

In [ ]:
print(F'The percentage of foreground gaalxies identified as background galaxies in cat_photoz dataset is {per_cent_cat_photoz:.2f} %')
print(F'The percentage of foreground gaalxies identified as background galaxies in cat_probability dataset is {per_cent_cat_probability:.2f} %')

This selection effect has to be taken in account in the computation of the excess surface density, as the fraction of background galaxies have mean photometric redshift below the redshift threeshold for the weak lensing analysis si non-zero, and member galaxies are identified as background galaxies.

# 4. Compute the excess surface density

### Estimator of the excess surface density

- The maximum likelihood estimator of the excess surface density in the radial bin $[R, R + \Delta R[$ is defined by $$\widehat{\Delta\Sigma}_+(R) = \frac{1}{\sum\limits_{s = 1} w_{s}}
     \sum\limits_{s= 1}w_{s}\langle\Sigma_{{\rm crit}}(z_s, z_l)^{-1}\rangle^{-1}\epsilon_+^{s}$$ where $\epsilon_+^{s}$ is the tangential ellipticity of the background galaxy with index $s$ (source) relative to the dark matter halo position (lens).
     
- The critical surface mass density expresses as $\Sigma_{{\rm crit}}(z_s, z_l) = \frac{c^2}{4 \pi G} \frac{D_A(z_s)}{D_A(z_l) D_A(z_s, z_l)}$, where $D_A(z_l), D_A(z_s)$ are respectively the angular diameter distance to the lens and to the source in physical units, and $D_A(z_s, z_l)$ is the angular diameter distance between the lens and the source.


- and the average $\langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle$ is defined as $\langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle = \int_{z_l + 0.1}^{+\infty} d z_s\ p_{\rm photoz}(z_s)\Sigma_{\rm crit}(z_s,z_l)^{-1}$, where $p_{\rm photoz}$ is the photometric probability density function for the background galaxy with index $s$.

### Weights $w_s$

The quantities $w_{s}$ are the weights that maximise the sigmnal-to-noise ratio of the excess surface density estimator. They downweight the galaxies that are close in redshift to the cluster (where the shape distorsion is small). It also indicates the lack of informations on both redshift and shape reconstruction for each background galaxies. 

- In the case where there is no error on the shape measurement (in the case of cosmoDC2), the weight $w_{s}$ writes $w_{s} = \langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle^{2}$,

- In the case of true redshift $z_s$, the PDF reduces to a Dirac function centered at $z_s$, giving the average $\langle\Sigma_{\rm crit}(z_l,z_s)^{-1}\rangle = \Sigma_{\rm crit}(z_l,z_s)^{-1} $ and the weight $w_{s} = \Sigma_{\rm crit}(z_s,z_l)^{-2}$,


In this notebook, we compute the excess surface density in 3 different cases, using different redshift informations:

- true backgroud galaxy redshift (computation of $w_{ls}^{\rm true}$)
- photometrical probability density function ($w_{ls}^{\rm pdf}$)
- mean photometric redshift ($w_{ls}^{\rm mean}$)

We will refer to the probability $p_{s} = \frac{w_{s}}{\sum_{s'} w_{s'}}$ in each case.

In [ ]:
cosmo = clmm.Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)

In [ ]:
sigma_crit_1 = np.array(1/cosmo.eval_sigma_crit(z_min, z_bins))

In [ ]:
m = np.zeros([len(cat_complete), len(z_bins)])

In [ ]:
for i in range(len(cat_complete)):
    m[i,:] = np.array(cat_complete['photoz_pdf'][i])

In [ ]:
unormed_integral = scipy.integrate.simps(m * sigma_crit_1, x = z_bins, axis = 1)
norm = scipy.integrate.simps(m, x = z_bins, axis = 1)

In [ ]:
ws_pdf = (unormed_integral/norm)**2
ws_true = ( 1. /cosmo.eval_sigma_crit(z_min, cat_complete['redshift']) )**2
ws_mean = ( 1. /cosmo.eval_sigma_crit(z_min, cat_complete['photoz_mean']) )**2

In [ ]:
ws_random = []
for pdf in cat_complete['photoz_pdf']:
    ws_random.append(np.random.choice(1./cosmo.eval_sigma_crit(z_min, z_bins), p = pdf/np.sum(pdf))**2)
ws_random = np.array(ws_random)

In [ ]:
ws_with_photozs = [ws_pdf, ws_mean, ws_random]
label = [r'$p_{s}^{\rm pdf}$', 
         r'$p_{s}^{\rm mean}$', 
         r'$p_{s}^{\rm random}$']
title = []

x = np.linspace(0, 7, 100)
fig, axs = plt.subplots(1, 3, figsize = (25,7))
for i in range(3):
    
    axs[i].scatter(cat_complete['redshift'], ws_with_photozs[i]/np.sum(ws_with_photozs[i]), s = 1, c = 'r', label = label[i])
    axs[i].scatter(cat_complete['redshift'], ws_true/np.sum(ws_true), s = 1, c = 'k')
    axs[i].vlines(z_cl, 0, 1, color = 'k', linestyle = '--', label = 'cluster redshift', zorder = 1000)
    axs[i].set_ylabel(label[i], fontsize = 25)
    axs[i].set_xlabel(r'$z_{\rm cosmoDC2}$', fontsize = 25)
    axs[i].tick_params(axis='both', which = 'major', labelsize= 15, zorder = 0)
    axs[i].legend(fontsize = 15, loc = 'lower right', frameon = True, framealpha = 1)
    axs[i].set_ylim(0,max(ws_with_photozs[0]/np.sum(ws_with_photozs[0])))

The normalized weight $p_s$ downweight galaxies with that are close in redshift tothe cluster (black curves). For the trhee cases, the weights $p_s$ scatter arround the black curves below $z \approx 1.5$, beyond this redshift, the eccurences of catastrophic photometric redshift reconstruction increases and the deviation to the black lines is larger.

In [ ]:
cat_complete['ws_true'] = np.array(ws_true)
cat_complete['ws_pdf'] = np.array(ws_pdf)
cat_complete['ws_mean'] = np.array(ws_mean)
cat_complete['ws_random'] = np.array(ws_random)

In [ ]:
data_complete = clmm.GCData(cat_complete)
cl_complete = clmm.GalaxyCluster('DM_halo', ra_cl, dec_cl, z_cl, data_complete)

We compute the tangential ellipticity for each background galaxy relative to the dark matter halo center.

In [ ]:
cl_complete.compute_tangential_and_cross_components(
                                    shape_component1='e1', shape_component2='e2', 
                                    tan_component='shear_t', cross_component='shear_x',
                                    geometry="flat")
cl_complete.galcat['r'] = cosmo.eval_da(cl_complete.z)*cl_complete.galcat['theta']

We also create a compltete background galaxy catalog with additive redshift cut by removing galaxies with true redshift beyond the threeshold. 

In [ ]:
data_true = clmm.GCData(cl_complete.galcat[cl_complete.galcat['redshift'] > z_min])
cl_true = clmm.GalaxyCluster('DM_halo', ra_cl, dec_cl, z_cl, data_true)

We now compute the excess surface density profile by binning the backgroud galaxy catalog in radial bins and computing the mean excess surface density in each bins using the above equation. 
- We define a set of bin edges,
- We compute the average excess surface density in each bins as described above for the 4 cases,
- We compute the errorbars as the standard deviation in each bin.

In [ ]:
bin_edges = clmm.make_bins(1,  6, nbins=10, method='evenlog10width')
radial_bin = [[bin_edges[i], bin_edges[i + 1]] for i in range(len(bin_edges)-1)]

In [ ]:
def make_radial_profile(catalog = 1, radius = '', tangential_ellipticity = '', weight = '', radial_bin = 1):

    profile = Table(names=['r', 'DS', 'DS_err'])
    
    for i, r_bin in enumerate(radial_bin):
        
        mask = (catalog.galcat[radius] > r_bin[0])*(catalog.galcat[radius] <= r_bin[1])*(catalog.galcat[weight] != 0)

        data_cut = catalog.galcat[mask]
        
        r_mean = np.mean(data_cut['r'])

        ds = np.average(data_cut[tangential_ellipticity]*data_cut[weight]**(-0.5), 
                             weights = data_cut[weight])
        
        #We compute the standard deviation of the excess surface density
        
        ds_rms = np.sqrt(np.average((data_cut['shear_t']*data_cut[weight]**(-0.5) - ds)**2, 
                                         weights = data_cut[weight]))/np.sqrt(len(data_cut))

        profile.add_row([r_mean, ds, ds_rms])
        
    return profile

### Effect of weight computation
We compute the excess surface density in 3 different cases, using different redshift informations and usong exclusively the `cat_true` catalog:

- true backgroud galaxy redshift (computation of $w_{ls}^{\rm true}$)
- photometrical probability density function ($w_{ls}^{\rm pdf}$)
- mean photometric redshift ($w_{ls}^{\rm mean}$)
- random sample redshift over the pdf ($w_{ls}^{\rm random}$)

In [ ]:
#true redhift
profile_true = make_radial_profile(catalog = cl_true, radius = 'r', 
                                   tangential_ellipticity = 'shear_t', 
                                   weight = 'ws_true', 
                                   radial_bin = radial_bin)

# using full photometric PDF
profile_pdf = make_radial_profile(catalog = cl_true, radius = 'r', 
                                  tangential_ellipticity = 'shear_t', 
                                  weight = 'ws_pdf', 
                                  radial_bin = radial_bin)

# using mean photometric redshift
profile_mean = make_radial_profile(catalog = cl_true, radius = 'r', 
                                   tangential_ellipticity = 'shear_t', 
                                   weight = 'ws_mean', 
                                   radial_bin = radial_bin)

# using random sample of the photometric PDF
profile_random = make_radial_profile(catalog = cl_true, radius = 'r', 
                                     tangential_ellipticity = 'shear_t', 
                                     weight = 'ws_random', 
                                     radial_bin = radial_bin)

We use the CLMM DESC code to compute the 'fiducial' excess surface mass density, by using a NFW profile with halo mass equals to the FoF mass from the catalog. We choose a concentration of 4.

In [ ]:
cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.27-0.045, Omega_b0=0.045, Omega_k0=0.0)
moo = clmm.Modeling(massdef='mean', delta_mdef=200, halo_profile_model='nfw')
moo.set_cosmo(cosmo)
moo.set_concentration(4)
moo.set_mass(mass_cl)
r = np.logspace(np.log10(0.1), np.log10(20), 100)
DeltaSigma = moo.eval_excess_surface_density(r, z_cl)

In [ ]:
ylabel = r'$\hat{\Delta\Sigma}_+$ ' +'$[$' + r'${\rm M}$' + r'$_\odot\;$'+ r'${\rm Mpc}$'+r'$^{-2}$'r'$]$'
xlabel = r'$R\ [$' + r'${\rm Mpc}$' + r'$]$'
plt.figure(figsize = (10,7))
plt.grid(True, which="both", ls="-")
plt.rcParams['axes.linewidth'] = 2
plt.tick_params(axis='both', which = 'both', labelsize= 15)
deltalogr = 0.03
dr = deltalogr*profile_true['r']
plt.errorbar(profile_true['r']  - 1*dr, profile_true['DS']  , profile_true['DS_err']  , fmt = ' ', marker = 'o', label = r'with $w_{ls}^{\rm true}$')
plt.errorbar(profile_pdf['r']   + 0*dr, profile_pdf['DS']   , profile_pdf['DS_err']   , fmt = ' ', marker = 'o', label = r'with $w_{ls}^{\rm pdf}$')
plt.errorbar(profile_mean['r']  + 1*dr, profile_mean['DS']  , profile_mean['DS_err']  , fmt = ' ', marker = 'o', label = r'with $w_{ls}^{\rm mean}$')
plt.errorbar(profile_random['r']+ 2*dr, profile_random['DS'], profile_random['DS_err'], fmt = ' ', marker = 'o', label = r'with $w_{ls}^{\rm random}$')
plt.plot(r, DeltaSigma,'-k', label = 'fiducial')
plt.xscale('log')
plt.xlim(1,10)
plt.ylim(-0.2e14,0.8e14)
plt.xlabel(xlabel, fontsize = 20)
plt.ylabel(ylabel, fontsize = 20)
plt.legend(frameon = False, fontsize = 20, loc = 'upper right')
plt.title('Excess surface density', fontsize = 20)
plt.show()

Below, we plot the bias :
    $$ bias = \frac{\hat{\Delta\Sigma}^{\rm (i)} - \hat{\Delta\Sigma}^{\rm true}}{\hat{\Delta\Sigma}^{\rm true}}$$
- where (i) denotes the different cases
- $\hat{\Delta\Sigma}^{\rm true}$ is the profile using only foreground galaxies and true redshifts

In [ ]:
plt.figure(figsize = (10,7))
plt.grid(True, which="both", ls="-")
plt.rcParams['axes.linewidth'] = 2
plt.tick_params(axis='both', which = 'both', labelsize= 15)
plt.errorbar(profile_pdf['r']   + 0*dr, -100*(profile_true['DS'] - profile_pdf['DS'])/profile_true['DS']    , fmt = '-', marker = 'o', label = r'with $w_{ls}^{\rm pdf}$')
plt.errorbar(profile_mean['r']  + 1*dr, -100*(profile_true['DS'] - profile_mean['DS'])/profile_true['DS']   , fmt = '-', marker = 'o', label = r'with $w_{ls}^{\rm mean}$')
plt.errorbar(profile_random['r']+ 2*dr, -100*(profile_true['DS'] - profile_random['DS'])/profile_true['DS'] , fmt = '-', marker = 'o', label = r'with $w_{ls}^{\rm random}$')
plt.xscale('log')
plt.xlabel(xlabel, fontsize = 20)
plt.ylabel(f'bias $(\%)$', fontsize = 20)
plt.legend(frameon = False, fontsize = 13)
plt.title('bias to true redshift case', fontsize = 20)
plt.show()

## Contamination effect

We create two catalogs based on different background galaxy selection criteria:
- `cl_photoz` with cut =  $\langle z \rangle > z_{DM} + 0.1$
- `cl_prpbability` with cut : $P(z > z_{\rm DM} + 0.1)  > 0.95$

In [ ]:
data_photoz = clmm.GCData(cl_complete.galcat[cl_complete.galcat['photoz_mean'] > z_min])
cl_photoz = clmm.GalaxyCluster('DM_halo', ra_cl, dec_cl, z_cl, data_photoz)

In [ ]:
data_probability = clmm.GCData(cl_complete.galcat[cl_complete.galcat['background_probability'] > 0.95])
cl_probability = clmm.GalaxyCluster('DM_halo', ra_cl, dec_cl, z_cl, data_probability)

In [ ]:
def contamination(catalog = 1, radius = '', radial_bin = 1):

    profile = Table(names=['r', 'count_foreground', 'count_member'])
    
    for i, r_bin in enumerate(radial_bin):
        
        mask_foreground = (catalog.galcat['redshift'] < z_cl)
        
        mask_radius = (catalog.galcat[radius] > r_bin[0])*(catalog.galcat[radius] <= r_bin[1])

        data_cut = catalog.galcat[mask_radius]
        
        data_cut_foreground = catalog.galcat[mask_radius*mask_foreground]
        
        r_mean = np.mean(data_cut['r'])
        
        data_member = catalog.galcat['galaxy_id'][mask_radius][np.isin(catalog.galcat['galaxy_id'][mask_radius],member_gal_catalog['galaxy_id'])]

        profile.add_row([r_mean, len(data_cut_foreground)/len(data_cut), len(data_member)/len(data_cut)])
        
    return profile

In [ ]:
bin_edges_ = clmm.make_bins(0.1,  6, nbins=10, method='evenlog10width')
radial_bin_ = [[bin_edges_[i], bin_edges_[i + 1]] for i in range(len(bin_edges_)-1)]
radial_middle = np.mean(radial_bin_, axis = 1)
contamination_photoz = contamination(catalog = cl_photoz, radius = 'r', 
                                  radial_bin = radial_bin_)
contamination_probability = contamination(catalog = cl_probability, radius = 'r', 
                                  radial_bin = radial_bin_)

In [ ]:
plt.figure(figsize = (10,7))
plt.rcParams['axes.linewidth'] = 2
plt.tick_params(axis='both', which = 'both', labelsize= 15)
plt.plot(radial_middle, contamination_photoz['count_foreground'],'b', label = 'cl_photoz : foreground galaxies')
plt.plot(radial_middle,  contamination_photoz['count_member'],'--b', label = 'cl_photoz : member galaxies')
plt.plot(radial_middle, contamination_probability['count_foreground'],'r', label = 'cl_probability : foreground galaxies')
plt.plot(radial_middle,  contamination_probability['count_member'],'--r', label = 'cl_probability : member galaxies')
plt.xscale('log')
plt.xlabel(xlabel, fontsize = 20)
plt.ylabel(f'fraction of galaxies in each radial bin', fontsize = 20)
plt.legend(frameon = False, fontsize = 17)
plt.show()

In [ ]:
profile_photoz = make_radial_profile(catalog = cl_photoz, radius = 'r', 
                                  tangential_ellipticity = 'shear_t', 
                                  weight = 'ws_pdf', 
                                  radial_bin = radial_bin)

In [ ]:
profile_probability = make_radial_profile(catalog = cl_probability, radius = 'r', 
                                  tangential_ellipticity = 'shear_t', 
                                  weight = 'ws_pdf', 
                                  radial_bin = radial_bin)

In [ ]:
ylabel = r'$\hat{\Delta\Sigma}_+$ ' +'$[$' + r'${\rm M}$' + r'$_\odot\;$'+ r'${\rm Mpc}$'+r'$^{-2}$'r'$]$'
xlabel = r'$R\ [$' + r'${\rm Mpc}$' + r'$]$'
plt.figure(figsize = (10,7))
plt.grid(True, which="both", ls="-")
plt.rcParams['axes.linewidth'] = 2
plt.tick_params(axis='both', which = 'both', labelsize= 15)
deltalogr = 0.03
dr = deltalogr*profile_true['r']

plt.errorbar(profile_true['r']   + 0*dr, profile_true['DS']   , profile_true['DS_err']   , fmt = ' ', marker = 'o', label = r'true - cut $z_{\rm cosmoDC2} > z_{DM} + 0.1$')
plt.errorbar(profile_photoz['r']   + 1*dr, profile_photoz['DS']   , profile_photoz['DS_err']   , fmt = ' ', marker = 'o', label = r'photoz - cut $\langle z \rangle > z_{DM} + 0.1$')
plt.errorbar(profile_probability['r']   +2*dr, profile_probability['DS']   , profile_probability['DS_err']   , fmt = ' ', marker = 'o', label = r'photoz - cut : $P(z > z_{\rm DM} + 0.1)  > 0.95$')
plt.plot(r, DeltaSigma,'-k', label = 'fiducial')
plt.xscale('log')
plt.xlim(1,10)
plt.ylim(-0.2e14,0.8e14)
plt.xlabel(xlabel, fontsize = 20)
plt.ylabel(ylabel, fontsize = 20)
plt.legend(frameon = False, fontsize = 15, loc = 'upper right')
plt.title('Excess surface density', fontsize = 20)
plt.show()